In [425]:
# py -m pip install --upgrade pip setuptools wheel
# py -m pip install catboost
# py -m pip install ipywidgets
# py -m pip install scikit-learn

import pandas as pd
import numpy as np

from catboost import CatBoostClassifier

from sklearn.preprocessing import LabelEncoder # Кодирование категориальных данных
from sklearn.feature_selection import SelectKBest # Выбор признаков
from sklearn.feature_selection import chi2 # Выбор признаков по Хи квадрат

from sklearn.model_selection import train_test_split # Деление выборки на тестовые и тренировочные данные

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Критерий качества, точности

In [426]:
# Генерируем тестовые данные и сохраняем в CSV файл
def generate_students(subjects: int, count: int):
  objects = [f"subject_{num + 1}" for num in range(subjects)]
  grades = np.random.randint(0, 100, (count, subjects))
  data = pd.DataFrame(data = grades, columns = objects)
  data.sum(axis = 0)
  data['score_mean'] = np.round((data.sum(axis = 1) / subjects), 3)
  for index, row in data.iterrows():
    if (row['score_mean'] < 45):
      data.loc[index, 'score_text'] = 'удовлетворительно'
    elif (row['score_mean'] >= 45 and row['score_mean'] < 75):
      data.loc[index, 'score_text'] = 'хорошо'
    else:
      data.loc[index, 'score_text'] = 'отлично'
  return data

generate = generate_students(subjects = 6, count = 2000)
generate.to_csv('data.csv', index = False)
generate


,subject_1,subject_2,subject_3,subject_4,subject_5,subject_6,score_mean,score_text
0,63,37,36,37,13,12,33.000,удовлетворительно
1,4,90,25,65,4,93,46.833,хорошо
2,73,35,23,66,75,61,55.500,хорошо
3,75,37,65,99,78,92,74.333,хорошо
4,77,61,22,40,1,64,44.167,удовлетворительно
...,...,...,...,...,...,...,...,...
1995,27,59,85,29,6,26,38.667,удовлетворительно
1996,43,9,51,85,38,82,51.333,хорошо
1997,31,52,11,49,35,65,40.500,удовлетворительно
1998,39,58,13,17,33,16,29.333,удовлетворительно


In [427]:
# Загружаем данные из CSV файла
data = pd.read_csv('data.csv')
data

,subject_1,subject_2,subject_3,subject_4,subject_5,subject_6,score_mean,score_text
0,63,37,36,37,13,12,33.000,удовлетворительно
1,4,90,25,65,4,93,46.833,хорошо
2,73,35,23,66,75,61,55.500,хорошо
3,75,37,65,99,78,92,74.333,хорошо
4,77,61,22,40,1,64,44.167,удовлетворительно
...,...,...,...,...,...,...,...,...
1995,27,59,85,29,6,26,38.667,удовлетворительно
1996,43,9,51,85,38,82,51.333,хорошо
1997,31,52,11,49,35,65,40.500,удовлетворительно
1998,39,58,13,17,33,16,29.333,удовлетворительно


In [428]:
# Определение признаков
X = data.drop(['score_mean', 'score_text'], axis = 1)
X

,subject_1,subject_2,subject_3,subject_4,subject_5,subject_6
0,63,37,36,37,13,12
1,4,90,25,65,4,93
2,73,35,23,66,75,61
3,75,37,65,99,78,92
4,77,61,22,40,1,64
...,...,...,...,...,...,...
1995,27,59,85,29,6,26
1996,43,9,51,85,38,82
1997,31,52,11,49,35,65
1998,39,58,13,17,33,16


In [429]:
# Определение целевой переменной
y = data['score_text']
y

0       удовлетворительно
1                  хорошо
2                  хорошо
3                  хорошо
4       удовлетворительно
              ...        
1995    удовлетворительно
1996               хорошо
1997    удовлетворительно
1998    удовлетворительно
1999              отлично
Name: score_text, Length: 2000, dtype: object

In [430]:
# Преобразовываем категориальный признак в числовой
label_encoder = LabelEncoder()
temp_y_encoded = label_encoder.fit_transform(y)
temp_y_encoded

array([1, 2, 2, ..., 1, 1, 0])

In [431]:
# Объеденяем категориальный признак текстовой с числовым и сохраняем в CSV файл
temp_y = y.values
y_new = pd.DataFrame(data = list(zip(temp_y, temp_y_encoded)), columns = ['y', 'y_encoded'])
y_new.to_csv('score_text_labels.csv', index=False)
y_new

,y,y_encoded
0,удовлетворительно,1
1,хорошо,2
2,хорошо,2
3,хорошо,2
4,удовлетворительно,1
...,...,...
1995,удовлетворительно,1
1996,хорошо,2
1997,удовлетворительно,1
1998,удовлетворительно,1


In [432]:
# Выбираем лучшие 3 признака с использованием критерия хи-квадрат
y = temp_y_encoded
selector = SelectKBest(chi2, k = 3)
X_best = selector.fit_transform(X, y)
X_best

array([[37, 13, 12],
       [65,  4, 93],
       [66, 75, 61],
       ...,
       [49, 35, 65],
       [17, 33, 16],
       [95, 70, 80]], dtype=int64)

In [433]:
# Разделяем датасет на тестовую и обучающую выборку
# 80% обучающей выборки, 20% тестовой
X = X_best
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 64)

In [434]:
# Обучаем модель CatBoostClassifier и сохраняем модель в файл
model = CatBoostClassifier(iterations = 100, depth = 4, learning_rate = 0.1)
model.fit(X_train, y_train)
model.save_model('people_university_model_eff.cbm')

0:	learn: 1.0179389	total: 951us	remaining: 94.2ms
1:	learn: 0.9454897	total: 2.17ms	remaining: 106ms
2:	learn: 0.8885463	total: 3.06ms	remaining: 99ms
3:	learn: 0.8456438	total: 3.89ms	remaining: 93.3ms
4:	learn: 0.8039621	total: 6.24ms	remaining: 119ms
5:	learn: 0.7683083	total: 7.15ms	remaining: 112ms
6:	learn: 0.7377631	total: 7.99ms	remaining: 106ms
7:	learn: 0.7139484	total: 8.82ms	remaining: 101ms
8:	learn: 0.6891367	total: 9.65ms	remaining: 97.6ms
9:	learn: 0.6692607	total: 10.5ms	remaining: 94.4ms
10:	learn: 0.6518988	total: 11.3ms	remaining: 91.5ms
11:	learn: 0.6349529	total: 12.1ms	remaining: 88.9ms
12:	learn: 0.6209520	total: 12.9ms	remaining: 86.6ms
13:	learn: 0.6096002	total: 13.8ms	remaining: 84.7ms
14:	learn: 0.5991502	total: 14.7ms	remaining: 83ms
15:	learn: 0.5886793	total: 15.5ms	remaining: 81.3ms
16:	learn: 0.5798358	total: 16.3ms	remaining: 79.6ms
17:	learn: 0.5711276	total: 17.5ms	remaining: 79.7ms
18:	learn: 0.5638252	total: 18.6ms	remaining: 79.2ms
19:	learn: 0.

In [435]:
# Загружаем мобель из файла
model_new = CatBoostClassifier()
model_new.load_model('people_university_model_eff.cbm')

In [436]:
# Делаем предсказания на тестовом наборе
y_pred = model_new.predict(X_test)
y_pred

array([[2],
       [2],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [1],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
    

In [437]:
# Оцениваем точность модели
accuracy_eff = accuracy_score(y_test, y_pred)
print(f"Правильность (accuracy) модели: {accuracy_eff}")

precision_eff = precision_score(y_test, y_pred, average='weighted')
print(f"Точность (precision) модели: {precision_eff}")

recall_eff = recall_score(y_test, y_pred, average='weighted')
print(f"Полнота (recall) модели: {recall_eff}")

f1_eff = f1_score(y_test, y_pred, average='weighted')
print(f"F1 мера модели: {f1_eff}")

Правильность (accuracy) модели: 0.76
Точность (precision) модели: 0.7362142857142857
Полнота (recall) модели: 0.76
F1 мера модели: 0.7456491228070175


In [442]:
def classifyStudent(grades_new):
  # Преобразование данных студента в DataFrame
  data_new = pd.DataFrame([grades_new])

  # Прогнозирование с использованием обученной модели (SelectBestK)
  predicted_category = model_new.predict(data_new)[0][0]

  # Преобразование обратно в текстовую категорию
  categories = ['отлично', 'удовлетворительно', 'хорошо']
  predict = categories[predicted_category]
  return predict


In [455]:
# Генерируем данные для прогноза
grades_new = np.random.randint(0, 100, 6)
grades_new

array([18, 21,  1, 87, 80, 35])

In [456]:
# Прогнозирование
predict = classifyStudent(grades_new)
print(f"Студент скорее всего получит (SelectBestK): {predict}")

Студент скорее всего получит (SelectBestK): удовлетворительно
